In [1]:
from glob import glob
import pandas as pd
from Bio import SeqIO
from paretoset import paretoset
import numpy as np
import datetime
import re

ADD GETORF INFO

In [2]:
files_dict = {filename.split('/')[-1].split('.')[0]: filename for filename in glob('results/orf_x_ref/*.csv')}
dfs = {key: pd.read_csv(file_path) for key, file_path in files_dict.items()}
getorf = '/home/tobamo/analize/project-tobamo/analysis/whole_orf/results/getorf_output_find1_min_360_max_3510_non_cellular.fasta'

In [3]:
with open(getorf, "r") as file:
    data = [record.description for record in SeqIO.parse(file, "fasta")]

getorf = pd.DataFrame({
    'orf_name': [item.split()[0].replace('=', '_') for item in data],
    'orf_start': [int(item.split('[')[1].split(' - ')[0]) for item in data],
    'orf_end': [int(item.split(' - ')[1].split(']')[0]) for item in data],
    'orientation': ['REVERSE' if "(REVERSE SENSE)" in item else 'FORWARD' for item in data]
})

In [4]:
for key, df in dfs.items():
    merged = df.merge(getorf, how='left', on='orf_name')
    # merged.to_csv(f'results/orf_x_ref_getorf_info/{key}_getorf_info.csv', index=False)
    dfs[key] = merged

FILTER AND MERGE

In [5]:
def calculate_pareto_set(group):
    group['on_pareto_front'] = paretoset(group[['identity_score', 'N/aln_len', 'gap_ratio']], sense=["max", "max", "min"])
    return group

In [6]:
for key, df in dfs.items():
    pareto = df.groupby('contig_name', group_keys=False).apply(calculate_pareto_set)
    dfs[key] = pareto

FIGURE OUT LIMITS FROM REFERENCE FILES

In [7]:
f_dict = {filename.split('/')[-1].split('.')[0]: filename for filename in glob('results/reference_limits/*.csv')}
ref_dfs = {key: pd.read_csv(file_path) for key, file_path in f_dict.items()}

In [8]:
final_tables = {}
metrics = {'identity_score': 'Identity Score', 'gap_ratio': 'Gap Ratio', 'N/aln_len': 'N/aln_len'}

# Iterate through each metric
for metric, label in metrics.items():
    max_scores_list = []
    min_scores_list = []
    
    # Iterate through each DataFrame
    for key, df in ref_dfs.items():
        max_scores_list.append(df.groupby('pair_type')[metric].max())
        min_scores_list.append(df.groupby('pair_type')[metric].min())

    max_scores_df = pd.concat(max_scores_list, axis=1, keys=ref_dfs.keys())
    min_scores_df = pd.concat(min_scores_list, axis=1, keys=ref_dfs.keys())

    final_tables[label] = pd.concat({
        f'Max {label}': max_scores_df.transpose(),
        f'Min {label}': min_scores_df.transpose()
    }, axis=1)

# Combine all final tables into a single table
final_table_combined = pd.concat(final_tables, axis=1)
final_table_combined

Identity Score                                   \
                 Max Identity Score               Min Identity Score   
pair_type                 og-tobamo tobamo-tobamo          og-tobamo   
rdrp_orf2_pareto           0.420635      1.000000           0.317734   
rdrp_orf1_pareto           0.353877      0.999105           0.283412   
cp_pareto                  0.407767      1.000000           0.184049   
mp_pareto                  0.321267      0.996212           0.202532   

                                   Gap Ratio                              \
                               Max Gap Ratio               Min Gap Ratio   
pair_type        tobamo-tobamo     og-tobamo tobamo-tobamo     og-tobamo   
rdrp_orf2_pareto      0.514019      0.035865      0.018367      0.003205   
rdrp_orf1_pareto      0.385379      0.050708      0.025907      0.028613   
cp_pareto             0.333333      0.079602      0.032680      0.021277   
mp_pareto             0.250951      0.061856      0.057047      0.028070   

                                   N/aln_len                              \
                               Max N/aln_len               Min N/aln_len   
pair_type        tobamo-tobamo     og-tobamo tobamo-tobamo     og-tobamo   
rdrp_orf2_pareto           0.0      0.996795           1.0      0.766600   
rdrp_orf1_pareto           0.0      0.839300           1.0      0.644068   
cp_pareto                  0.0      0.867021           1.0      0.453744   
mp_pareto                  0.0      0.899614           1.0      0.661932   

                                
                                
pair_type        tobamo-tobamo  
rdrp_orf2_pareto      0.841897  
rdrp_orf1_pareto      0.670753  
cp_pareto             0.746544  
mp_pareto             0.684366

In [9]:
# final_table_combined.to_excel('reference_limits_table.xlsx')

FILTER DFS

In [10]:
filtered_dfs = {}
pd.options.mode.chained_assignment = None

for key, df in dfs.items():
    name = key.replace('orf_x_ref', 'pareto')

    id_score_lim_sl = final_table_combined.loc[(f'{name}'), ('Identity Score', 'Min Identity Score', 'tobamo-tobamo')]
    naln_len_lim_sl = final_table_combined.loc[(f'{name}'), ('N/aln_len', 'Min N/aln_len', 'tobamo-tobamo')]
    gap_ratio_lim_sl = final_table_combined.loc[(f'{name}'), ('Gap Ratio', 'Max Gap Ratio', 'tobamo-tobamo')]


    id_score_lim_ss = final_table_combined.loc[(f'{name}'), ('Identity Score', 'Max Identity Score', 'og-tobamo')]
    naln_len_lim_ss = final_table_combined.loc[(f'{name}'), ('N/aln_len', 'Max N/aln_len', 'og-tobamo')]
    gap_ratio_lim_ss = final_table_combined.loc[(f'{name}'), ('Gap Ratio', 'Min Gap Ratio', 'og-tobamo')]

    def filter_rows(dataframe):
        filtered_df = dataframe[(dataframe['identity_score'] >= id_score_lim_sl) & (dataframe['N/aln_len'] >= naln_len_lim_sl) & (dataframe['gap_ratio'] <= gap_ratio_lim_sl)]
        filtered_df['tax_trust'] = (
            (filtered_df['identity_score'] >= id_score_lim_ss) &
            (filtered_df['N/aln_len'] >= naln_len_lim_ss) &
            (filtered_df['gap_ratio'] <= gap_ratio_lim_ss)
        )
        return filtered_df

    filtered_df = filter_rows(df)
    filtered_dfs[key] = filtered_df

edit col names, add prefixes

In [11]:
num = 0

for key, df in filtered_dfs.items():
    name = key.replace('_orf_x_ref', '')
    df = df.drop(labels='SRR', axis=1)
    rnm = df[df['on_pareto_front'] == True].copy()
    rnm[f'idscore>{num}'] = np.where(rnm['identity_score'] > num, True, False)
    rnm.rename(columns={col: f'{name}_{col}' for col in rnm.columns if col not in ['contig_name']}, inplace=True)
    filtered_dfs[key] = rnm

In [12]:
# merge dataframes
megamerge = filtered_dfs['rdrp_orf1_orf_x_ref'].merge(filtered_dfs['rdrp_orf2_orf_x_ref'], on='contig_name', how='outer').merge(filtered_dfs['mp_orf_x_ref'], on='contig_name', how='outer').merge(filtered_dfs['cp_orf_x_ref'], on='contig_name', how='outer')

In [13]:
megamerge['SRR'] = megamerge["contig_name"].str.extract(r"_([A-Za-z0-9]+)$")

In [14]:
pd.options.mode.chained_assignment = None

megamerge['contig_len'] = megamerge['contig_name'].apply(lambda x: int(re.search(r'length_(\d+)|len_(\d+)', x).group(1) or re.search(r'len_(\d+)', x).group(1)))
megamerge['orientation_match'] = megamerge[['rdrp_orf1_orientation', 'rdrp_orf2_orientation', 'mp_orientation', 'cp_orientation']].apply(lambda row: len(set(row.dropna())) == 1, axis=1)
megamerge['different_orfs'] = megamerge[['rdrp_orf1_orf_name', 'rdrp_orf2_orf_name', 'mp_orf_name', 'cp_orf_name']].apply(
    lambda row: 'different orfs' if len(set(row.dropna())) == len(row.dropna()) else 'check',
    axis=1
)

In [15]:
# set first cols
first_columns = ['SRR', 'contig_name','different_orfs', 'contig_len',  f'rdrp_orf1_idscore>{num}',  f'rdrp_orf2_idscore>{num}', f'mp_idscore>{num}', f'cp_idscore>{num}']

# Reorder the columns
mega = megamerge[first_columns + [col for col in megamerge.columns if col not in first_columns]]

In [16]:
current_date = datetime.datetime.now().strftime("%Y-%m-%d")
# mega.to_csv(f'results/orf_x_ref_megamerge_{current_date}_filtered.csv')

In [108]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# df = mega[['contig_name', 'rdrp_orf1_idscore>0.5', 'rdrp_orf2_idscore>0.5', 'mp_idscore>0.5', 'cp_idscore>0.5']]
# filtered = df.drop_duplicates().drop(columns=['contig_name'])

# # Stacking the DataFrame
# stacked_df = filtered.stack().reset_index()
# stacked_df.columns = ['index', 'column', 'value']

# # Specifying the order of x-axis labels
# order = ['rdrp_orf1_idscore>0.5', 'rdrp_orf2_idscore>0.5', 'mp_idscore>0.5', 'cp_idscore>0.5']

# # Plotting the count of each value in each column with sorted x-axis labels
# plt.figure(figsize=(10,5))
# ax = sns.countplot(data=stacked_df, x='column', hue='value', order=order)

# # Adding counts above bars
# for p in ax.patches:
#     height = p.get_height()  # Get the height of the bar
#     ax.annotate(f'{int(height)}',  # Annotation text
#                 (p.get_x() + p.get_width() / 2., height),  # Position
#                 ha='center', va='center',  # Horizontal and vertical alignment
#                 fontsize=10,  # Font size
#                 color='black',  # Color of the text
#                 xytext=(0, 5),  # Offset the text vertically by 5 units
#                 textcoords='offset points')  # Text coordinates relative to the point


# plt.xlabel('Columns')
# plt.ylabel('Count')
# plt.title('Count of True, False, and NaN Values in Each Column')
# plt.legend(title='Value', loc='upper right')
# plt.show()

In [17]:
mega.contig_name.nunique()

365

Separate movement-only and others for blastn priority

In [18]:
movement_only_mask = (
    mega['mp_idscore>0'] == True
) & (
    mega[['rdrp_orf1_idscore>0', 'rdrp_orf2_idscore>0', 'cp_idscore>0']].isna().all(axis=1)
)
others_mask = ~movement_only_mask

movement_only_contigs = mega.loc[movement_only_mask, 'contig_name'].unique().tolist()
other_contigs = mega.loc[others_mask, 'contig_name'].unique().tolist()

In [19]:
fasta_in = '/home/tobamo/analize/project-tobamo/analysis/palmscan/analysis_data/test_contigs_2024-04-09_non_cellular.fasta'
fasta_out = '/home/tobamo/analize/project-tobamo/analysis/palmscan/analysis_data/test_contigs_2024-04-09_non_cellular_.fasta'
with open(fasta_in, 'r') as fin, open(fasta_out, 'w') as fout:
    for record in SeqIO.parse(fin, 'fasta'):
        record.id = record.id.replace('=', '_')
        record.description=''
        SeqIO.write(record, fout, 'fasta')

Are all mega contigs in contigs_non_cellualr?

In [20]:
mega.contig_name.nunique()

365

In [21]:
contigs = SeqIO.to_dict(SeqIO.parse('/home/tobamo/analize/project-tobamo/analysis/palmscan/analysis_data/test_contigs_2024-04-09_non_cellular_.fasta', 'fasta'))
len(contigs.keys())

2393

In [22]:
# check if there are any missing contigs
x = [contigs[z] for z in contigs if z in mega.contig_name.unique()]
xx = [record.id for record in x]
missing = [el for el in mega.contig_name.unique() if el not in xx]
missing

[]

In [23]:
# Filter to keep only the contigs with IDs in the 'other_contigs' list
filtered_contigs = [contigs[contig_id] for contig_id in contigs if contig_id in other_contigs]
filtered_contigs_m = [contigs[contig_id] for contig_id in contigs if contig_id in movement_only_contigs]

In [25]:
len(filtered_contigs), len(filtered_contigs_m)

(138, 227)

In [33]:
with open('results/blast/other_contigs.txt', 'w') as fout:
    fout.writelines(f'{item}\n' for item in other_contigs)

with open('results/blast/movement_only_contigs.txt', 'w') as fout:
    fout.writelines(f'{item}\n' for item in movement_only_contigs)

In [26]:
# Path to the output FASTA file
output_fasta = f'/home/tobamo/analize/project-tobamo/analysis/whole_orf/results/blast/other_contigs_{len(filtered_contigs)}.fasta'

# Write the filtered contigs to the new FASTA file
with open(output_fasta, 'w') as output_handle:
    SeqIO.write(filtered_contigs, output_handle, 'fasta')

In [27]:
# Path to the output FASTA file
output_fasta = f'/home/tobamo/analize/project-tobamo/analysis/whole_orf/results/blast/movement_only_contigs_{len(filtered_contigs_m)}.fasta'

# Write the filtered contigs to the new FASTA file
with open(output_fasta, 'w') as output_handle:
    SeqIO.write(filtered_contigs_m, output_handle, 'fasta')

In [39]:
contigs = list(mega.contig_name.unique())

with open('results/blast/all_contigs_for_blast.txt', 'w') as fout:
    fout.writelines(f'{item}\n' for item in contigs)